# Module 4

Score rules. Please solve exercises worth at most 30 pts. For each correct execution you will receive an assigned number of points. Its sum will determine the final score.


1. Implement Diffie-Hellman key exchange on an ellptic curve (missing: 0 pts, implementation up to 10 pts)
2. Implement a simple block cipher (missing: 0 pts, implementation up to 10 pts)
3. Design a hybrid system (missing: 0 pts, implementation up to 10 pts)

In [5]:
import random

# 1. Implement Diffie-Hellman key exchange on an ellptic curve
Input: 
1. An elliptic curve $E:y^2=x^3+Ax+B$ over a finite field $\mathbb{F}_{p}$, $p$-prime number.
2. A point $P$ of order $q$ in $E(\mathbb{F}_{p})$.

Output: A shared secret point $Q$ common to Alice and Bob which is $Q=kP$, $k$ is secretely generated in the followin process:

Alice generates a random multiple $P_{A}=xP$ of the point $P$, $0<x<q$

Bob generates a random multiple $P_{B}=yP$ of the point $P$, $0<y<q$.

They exchange the points $P_{A}$ and $P_{B}$ and generate the multiple $Q=kP$, where $k=xy$.

Emulate such an exchange between Alice and Bob.

Generate a sequence of $128$ bits out of the point $Q$. This will be the key used in algorithm $3$.

In [6]:
def opp(px, py, P):
    return [px, -py % P]

def egcd(a, b):
    s0, s1, t0, t1 = 1, 0, 0, 1
    while b > 0:
        q, r = divmod(a, b)
        a, b = b, r
        s0, s1, t0, t1 = s1, s0 - q * s1, t1, t0 - q * t1
        pass
    return [s0, t0, a]

def inv(n, q):
    return egcd(n, q)[0] % q

def invPoint(EC, P):
    ret [P[0], egcd(P[1], EC[2])[0] % EC[2]]

def add(A, B, p, p1x, p1y, p2x, p2y):
        if p1x == 0 and p1y == 0: return [p2x, p2y]
        if p2x == 0 and p2y == 0: return [p1x, p1y]
        if p1x == p2x and (p1y != p2y or p1y == 0):
            return [0, 0]
        if p1x == p2x:
            l = (3 * p1x * p1x + A) * inv(2 * p1y, p) % p
            pass
        else:
            l = (p2y - p1y) * inv(p2x - p1x, p) % p
            pass
        x = (l * l - p1x - p2x) % p
        y = (l * (p1x - x) - p1y) % p
        return [x,y]
    
def order(EC, P):
    for i in range(1, EC[2] + 1):
        if MultPoint(EC, P, i) == [0, 0]:
            return i
        pass
    raise Exception("Invalid order")

def MultPoint(EC,P,n):
    res = [0,0]
    aux = P
    count = abs(n)
    while 0 < count:
        if count & 1 == 1:
            res = add(EC[0], EC[1], EC[2], res[0], res[1], aux[0], aux[1])
            pass
        count = count >> 1
        aux = add(EC[0], EC[1], EC[2], aux[0], aux[1], aux[0], aux[1])
        pass
    if(n < 0):
        res = opp(res[0], res[1], EC[2])
    return res

def generateRandomMultiplePoint(EC, P):
    q = order(EC, P)
    n = random.randint(1, q-1)
    return MultPoint(EC, P, n), n

def GetQ(EC, I):
    return MultPoint(EC, I[2], I[1])
    pass

def DiffieHellmanExchange(EC, P):
    #emulate this action with prints and return an actual point on the selected curve
    # Alice structure: PA, x, PB
    # Alice generates PA
    Alice = [[0,0], 0, [0,0]]
    Alice[0], Alice[1] = generateRandomMultiplePoint(EC, P)
    # Bob structure: PB, y, PA
    # Bob generates PB
    Bob = [[0,0], 0, [0,0]]
    Bob[0], Bob[1] = generateRandomMultiplePoint(EC, P)
    # Alice send PA to Bob
    Bob[2] = Alice[0]
    # Bob send PB to Alice
    Alice[2] = Bob[0]
    # Alice and Bob get Q
    QAlice = GetQ(EC, Alice)
    QBob = GetQ(EC, Bob)
    pass

# 2. Implement a simple block cipher

Design a simple block cipher which operates on a block of $32$ bits, called $B$ and in each round of the cipher performs the following actions:

A) SBox(B): SBox function can be any reversible function which transforms the block $B$ into another block $B'$ with 32 blocks.

B) RoundXOR(B,K): Applying XOR function to the block $B$ with the round key $K$.

C) Perm(B): Apply a permutation function which swaps the bits in the block $B$ in the following way: divide the block into four blocks of $8$ bits and then label the subblocks as $(1,2,3,4)$. Produce a new block $B'$ which has the structure $(3,1,4,2)$

Now your cipher should operate on the input block $B$ as follows: 

1. Produce round keys $K_1, K_2, K_3, K_4$ of size $32$ bits from the $128$ bit key generated in the algorithm

In [7]:
def ProduceRoundKey(inpK): #inpK has 128 bits, the output should be a list of four 32 bit keys
    ret = []
    aux = list(str(inpK))
    for i in aux:
        ret.append(i)
    return [ret[0:(int(len(aux)*0.25))], ret[(int(len(aux)*0.25)):(int(len(aux)*0.5))],
             ret[(int(len(aux)*0.5)):(int(len(aux)*0.75))], ret[(int(len(aux)*0.75)):(int(len(aux)))]]
    pass

def SBox(B):
    ret = []
    aux = [int(x) for x in str(B)] 
    for i in aux:
        ret.append((i+1)%10)
    return ''.join(ret)
    pass

def InvSBox(state):
    ret = []
    aux = list(state)
    for bit in aux:
        ret.append((i-1)%10)
    return ''.join(ret)
    pass

def RoundXOR(state,K):
    K_strings = [str(integer) for integer in K]
    K_string = "".join(K_strings)
    print(state)
    print(K_string)
    return state^int(K_string)
    pass

def Perm(state):
    #(3,1,4,2)
    aux1 = list(state)
    aux2 = [aux1[0:8], aux1[8,16], aux1[16, 24], aux1[24:32]]
    perm = [aux2[2], aux2[0], aux2[3], aux2[1]]
    return perm
    pass

def InvPerm(state):
    aux1 = list(state)
    aux2 = [aux1[0:8], aux1[8,16], aux1[16, 24], aux1[24:32]]
    invPerm = [aux2[1], aux2[3], aux2[0], aux2[2]]
    return invPerm
    pass
    

2. Coding procedure

In [8]:
def Encode(B,K): #K is a list of four 32 bit keys produced from the 128 bit key from alg. 1
    state=B
    for i in [0,1,2]:
        state=RoundXOR(state,K[i])
        state=SBox(state)
        state=Perm(state)

    state=RoundXOR(state,K[3])
    return state

2. Decoding procedure

Implement `InvPerm`, `InvSBox` - inverses of the appropriate functions.

In [9]:
def Decode(C,K):
    state=C
    state=RoundXOR(state,K[3])

    for i in [2,1,0]:
        state=InvPerm(state)
        state=InvSBox(state)
        state=RoundXOR(state,K[i])
    return state

# 3. Design a hybrid system

Design a system in which the two parties Alice and Bob generate a common $128$ bit key `inpK` using procedure 1. Next, they establish a communication protocol in which they send blocks of $32$ bit size which are encoded using the procedure `Encode(B,K)` where `K` is a list of four $32$ bit blocks produced from `inpK`. 

Alice is encoding a message to Bob with `C=Encode(B,K)` and Bob decrypts it using `B=Decode(C,K)`. 

Emulate this system with prints and show how the message is being transferred.

You can allow the user to send text on the input, process this plaintext into chunks of size $32$ bits, encrypt it, send it, decode it and reconstruct the plaintext from the decoded chunks (design your own procedure for that).



In [10]:
inpK = random.getrandbits(128)
print("Input: " + str(inpK))
K = ProduceRoundKey(inpK)
print("Keys: ")
print(K)
B = random.getrandbits(32)
print("Block before encoding: ")
print(B)
print("Alice is encrypting the plaintext")
C = Encode(B, K)
print("Encoded plaintext: ")
print(C)
print("Sending to Bob..")
print("Bob is decrypting..")
D = Decode(C, K)
print("Decoded plaintext: ")
print(D)

Input: 224483935309124472368133306878661078574
Keys: 
[['2', '2', '4', '4', '8', '3', '9', '3', '5'], ['3', '0', '9', '1', '2', '4', '4', '7', '2', '3'], ['6', '8', '1', '3', '3', '3', '0', '6', '8', '7'], ['8', '6', '6', '1', '0', '7', '8', '5', '7', '4']]
Block before encoding: 
111768771
Alice is encrypting the plaintext
111768771
224483935


TypeError: sequence item 0: expected str instance, int found